## Este notebook versa sobre Dynamic Programming & Data Science
### Trabalha-se com identificação de palavras em texto

In [108]:
# importação de bibliotecas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score
import numpy as np

In [109]:
# Função de Dynamic Programming para selecionar as palavras mais relevantes
def caminho_otimo_tfidf(vetor):
    n = len(vetor)
    dp = np.zeros(n)
    dp[0] = vetor[0]
    indices = [0]

    for i in range(1, n):
        if vetor[i] > dp[i - 1]:
            dp[i] = vetor[i]
            indices.append(i)
        else:
            dp[i] = dp[i - 1]
            
    return [feature_names[i] for i in indices if vetor[i] > 0]

In [110]:
# parametro tecnico para usar no algoritmo SVM na classficacao
param_grid = [0.1, 1, 10, 100, 1000]

In [111]:
# Conjunto de textos - exemplos
texts = [
    "A inflação no Brasil teve um aumento considerável nos últimos meses, impactando o preço dos alimentos, combustíveis e serviços essenciais. Esse aumento afeta diretamente o poder de compra da população e pressiona o Banco Central a reajustar as taxas de juros para conter o avanço da inflação.",
    "As próximas eleições municipais estão gerando grande expectativa entre os eleitores, especialmente em relação aos candidatos que defendem mudanças nas políticas de educação e saúde. As propostas dos candidatos são debatidas amplamente nas redes sociais, e a população está cada vez mais engajada.",
    "A Black Friday está chegando e promete oferecer grandes descontos em produtos eletrônicos, moda e artigos para o lar. No entanto, é importante que os consumidores fiquem atentos às fraudes e verifiquem se os preços realmente valem a pena antes de realizar uma compra.",
    "O campeonato de futebol deste ano está sendo um dos mais disputados, com várias equipes de ponta competindo pela liderança. Os torcedores lotam os estádios e acompanham cada partida, torcendo para que seu time conquiste o título.",
    "O vestibular é um momento decisivo para milhares de estudantes que se preparam para entrar na universidade. Com a concorrência alta, muitos alunos intensificam os estudos e participam de cursinhos preparatórios para garantir uma boa colocação."
]
labels = ["inflação", "eleições", "black friday", "futebol", "vestibular"]

In [112]:
# Extraindo features de palavras usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts).toarray()
feature_names = vectorizer.get_feature_names_out()

In [113]:
# Seleção das palavras mais relevantes para cada texto
palavras_relevantes = [caminho_otimo_tfidf(x) for x in X]

print("Palavras relevantes por texto:", palavras_relevantes)

Palavras relevantes por texto: [['afeta', 'aumento'], ['amplamente', 'as', 'candidatos'], ['antes', 'os'], ['acompanham', 'os'], ['alta', 'para']]


In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [115]:
melhor_f1_score = 0
melhor_C = 1

for C in param_grid:
    svm = SVC(kernel='linear', C=C)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    f1_score_atual = f1_score(y_test, y_pred, average='weighted')
    
    if f1_score_atual > melhor_f1_score:
        melhor_f1_score = f1_score_atual
        melhor_C = C

In [116]:
print("Melhor parâmetro C:", melhor_C)

Melhor parâmetro C: 1


In [117]:
best_svm = SVC(kernel='linear', C=melhor_C)
best_svm.fit(X_train, y_train)
y_pred = best_svm.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

    eleições       1.00      0.00      0.00       1.0
     futebol       0.00      1.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.50      0.50      0.00       1.0
weighted avg       1.00      0.00      0.00       1.0

